Eodag advanced features
=====================

We present eodag advanced features for searching and downloading products.

Let's start by setting your personal credentials to access [sobloo service](https://sobloo.eu/) by filling your API key bellow:

In [1]:
import os
from pprint import pprint
# os.environ["EODAG__SOBLOO__AUTH__CREDENTIALS__APIKEY"] = "PLEASE_CHANGE_ME"

Change download directory (outputs_prefix) and extraction setting (True to False) using Environment variables. Every parameter from user configuration can be changed this way. The environment variable must respect the pattern `KEY1__KEY2__[...]__KEYN` for overriding an entry like:
```yaml
KEY1:
    KEY2:
        [...]
            KEYN: value
```

In [2]:
# os.environ["EODAG__SOBLOO__DOWNLOAD__OUTPUTS_PREFIX"] = "/tmp/eodag-data"
# os.environ["EODAG__SOBLOO__DOWNLOAD__EXTRACT"] =  "false"

Then we create a workspace directory where all our files and configuration will live:

In [3]:
from eodag.api.core import EODataAccessGateway
from eodag.utils.logging import setup_logging

workspace = 'eodag_workspace'
if not os.path.isdir(workspace):
    os.mkdir(workspace)

# To have some basic feedback on what eodag is doing, we configure logging to output minimum information
# setup_logging(verbose=1)
dag = EODataAccessGateway()

Let's check the preferred provider and its priority set in user configuration, then change the preferred provider to sobloo

In [4]:
print(dag.get_preferred_provider())
dag.set_preferred_provider("sobloo")
print(dag.get_preferred_provider())

('peps', 1)
('sobloo', 2)


Now let's search for Sentinel-2 L1C products in the South of France, acquired in the last 30 days.

As complementary search parameters, let us filter at 10% of maximum cloud cover (cloudCover is an OGC standard parameter), and only get ONLINE products (parameter syntax specific to sobloo provider):

In [5]:
from datetime import date, timedelta

product_type = 'S2_MSI_L1C'
extent = {
    'lonmin': -1.999512,
    'lonmax': 4.570313,
    'latmin': 42.763146,
    'latmax': 46.754917
}

products, estimated_total_nbr_of_results = dag.search(
    productType=product_type,
    start=(date.today() - timedelta(days=30)).isoformat(),
    end=date.today().isoformat(),
    box=extent,
    cloudCover=10,
    **{"state.services.download": "internal"}
)
pprint(products)

[EOProduct(id=S2A_MSIL1C_20201213T104441_N0209_R008_T31TEM_20201213T125210, provider=sobloo),
 EOProduct(id=S2B_MSIL1C_20201217T111359_N0209_R137_T30TWR_20201217T132006, provider=sobloo),
 EOProduct(id=S2B_MSIL1C_20201217T111359_N0209_R137_T30TWQ_20201217T150041, provider=sobloo),
 EOProduct(id=S2B_MSIL1C_20201217T111359_N0209_R137_T30TXQ_20201217T150041, provider=sobloo),
 EOProduct(id=S2B_MSIL1C_20201218T104349_N0209_R008_T31TEL_20201218T115054, provider=sobloo),
 EOProduct(id=S2B_MSIL1C_20201218T104349_N0209_R008_T31TCH_20201218T115054, provider=sobloo),
 EOProduct(id=S2B_MSIL1C_20201218T104349_N0209_R008_T31TDL_20201218T115054, provider=sobloo),
 EOProduct(id=S2B_MSIL1C_20210103T110349_N0209_R094_T30TYP_20210103T121021, provider=sobloo),
 EOProduct(id=S2A_MSIL1C_20210108T110431_N0209_R094_T31TCK_20210108T131314, provider=sobloo),
 EOProduct(id=S2A_MSIL1C_20210108T110431_N0209_R094_T30TXS_20210108T131314, provider=sobloo),
 EOProduct(id=S2A_MSIL1C_20201213T104441_N0209_R008_T31TEH_2

Plot search extent and found product on a map 

In [6]:
import ipyleaflet as ipyl

m = ipyl.Map(center=[43.6, 1.5], zoom=5)
search_layer = ipyl.Rectangle(
    bounds=((extent["latmin"], extent["lonmin"]), (extent["latmax"], extent["lonmax"])),
    color="green"
)
products_layer = ipyl.GeoJSON(data=products.as_geojson_object(), hover_style={'fillColor': 'yellow'})
m.add_layer(search_layer)
m.add_layer(products_layer)
m

Map(center=[43.6, 1.5], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_…

Use a cruncher on search results to filter the products fully contained in the search bounding box

In [13]:
from eodag.plugins.crunch.filter_overlap import FilterOverlap

filtered_products = products.crunch(FilterOverlap({"minimum_overlap": 100}))

In [14]:
pprint(filtered_products)

[EOProduct(id=S2A_MSIL1C_20201213T104441_N0209_R008_T31TEM_20201213T125210, provider=sobloo),
 EOProduct(id=S2B_MSIL1C_20201217T111359_N0209_R137_T30TWR_20201217T132006, provider=sobloo),
 EOProduct(id=S2B_MSIL1C_20201217T111359_N0209_R137_T30TWQ_20201217T150041, provider=sobloo),
 EOProduct(id=S2B_MSIL1C_20201217T111359_N0209_R137_T30TXQ_20201217T150041, provider=sobloo),
 EOProduct(id=S2B_MSIL1C_20201218T104349_N0209_R008_T31TEL_20201218T115054, provider=sobloo),
 EOProduct(id=S2B_MSIL1C_20201218T104349_N0209_R008_T31TCH_20201218T115054, provider=sobloo),
 EOProduct(id=S2B_MSIL1C_20201218T104349_N0209_R008_T31TDL_20201218T115054, provider=sobloo),
 EOProduct(id=S2B_MSIL1C_20210103T110349_N0209_R094_T30TYP_20210103T121021, provider=sobloo),
 EOProduct(id=S2A_MSIL1C_20210108T110431_N0209_R094_T31TCK_20210108T131314, provider=sobloo),
 EOProduct(id=S2A_MSIL1C_20210108T110431_N0209_R094_T30TXS_20210108T131314, provider=sobloo),
 EOProduct(id=S2A_MSIL1C_20201213T104441_N0209_R008_T31TEH_2

Add filtered products to previous map

In [9]:
filtered_products_layer = ipyl.GeoJSON(
    data=filtered_products.as_geojson_object(), 
    hover_style={'fillColor': 'yellow'},
    style={"color":"red"}
)
m.add_layer(filtered_products_layer)

Display product id on hover, to help chosing which to download. Click on a product to add it to the donwload list.

In [10]:
import ipywidgets as ipyw
from eodag.api.product import EOProduct
from eodag.api.search_result import SearchResult

label = ipyw.Label(layout=ipyw.Layout(width='100%'))

chosen_products = SearchResult([])

def click_handler(feature=None, **kwargs):
    if feature and feature not in chosen_products:
        chosen_products.append(EOProduct.from_geojson(feature))
        # register product downloader
        chosen_products[-1].register_downloader(products[0].downloader, products[0].downloader_auth)
        # update map
        chosen_products_layer = ipyl.GeoJSON(
            data=chosen_products.as_geojson_object(), 
            hover_style={'fillColor': 'yellow'},
            style={"color":"yellow"}
        )
        if chosen_products_layer in m.layers:
            m.remove_layer(chosen_products_layer)
        m.add_layer(chosen_products_layer)
        
filtered_products_layer.on_click(click_handler)

def hover_handler(id=None, **kwargs):
    label.value = id
filtered_products_layer.on_hover(hover_handler)

ipyw.VBox([m, label])

Download the chosen products (yellow ones on the map).
If the products are offline, they will be ordered and eodag will retry to download them every 2 minutes, and keep retrying for 20 minutes (default times). Let us change these times to 30s and 5 minutes

In [13]:
from eodag.utils import NotebookProgressCallback

if chosen_products:
    paths = dag.download_all(
        chosen_products, 
        progress_callback=NotebookProgressCallback(), 
        wait=0.5, 
        timeout=5
    )

This concludes the tutorial for the advanced usage of eodag.